In [1]:
import pandas as pd
import numpy as np
from collections import defaultdict
import scipy
import scipy.stats
from statsmodels.stats.multitest import multipletests


def filter_functions(callstacks, names):
  result = [[function for function in callstack if function in names] for callstack in callstacks]
  return [callstack for callstack in result if len(callstack) > 1]

class Capture:
  # is used to throw out the initialization period when the benchmark has not been started yet, but the 
  # timetime is excessively long
  def __init__(self, callstack_csv, frame_mean, frame_var, frame_ends, init_end_time): 
    self.frame_mean = frame_mean
    self.frame_var = frame_var / 1e6 / 1e6
    self.frame_ends = frame_ends
    self.callstack_csv = callstack_csv
    
    self.callstack_csv = self.callstack_csv[self.callstack_csv[:,1] > init_end_time,:]
    
    self.callstack_csv = self.callstack_csv[self.callstack_csv[:, 1].argsort()] # sort by timestamp
    self.frame_index_to_callstacks = self.get_frame_index_to_callstacks()
    self.total_callstacks = sum(len(s) for s in self.frame_index_to_callstacks.values())

  def filter_functions(self, names):
    filterred = {}
    for frame_index, callstacks in self.frame_index_to_callstacks.items():
      filterred[frame_index] = filter_functions(callstacks, names)
    self.frame_index_to_callstacks = filterred

  # returns a dict frame_index -> List[Callstack]
  def get_frame_index_to_callstacks(self): 
    frame_index_to_callstacks = {}
    frame_index = 0
    frame_end = self.frame_ends[frame_index]
    frame_index_to_callstacks[frame_index] = []
    callstack_names = [s.split('/') for s in  self.callstack_csv[:, 2]]
    timestamps = self.callstack_csv[:, 1]
    for i in range(self.callstack_csv.shape[0]):
      timestamp = timestamps[i]

      while timestamp > frame_end:
        if frame_index + 1 == self.frame_ends.shape[0]: 
          return frame_index_to_callstacks
        frame_index += 1
        frame_end = self.frame_ends[frame_index]
        frame_index_to_callstacks[frame_index] = [] 
      frame_index_to_callstacks[frame_index].append(callstack_names[i])
    return frame_index_to_callstacks

  def get_exclusive_count(self, function_name):
    exclusive_count = 0;
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        if (len(callstack) > 0 and callstack[0] == function_name):
          exclusive_count += 1
    return exclusive_count

  def avg_frame_time(self):
    return self.frame_mean / 1e6

  def total_callstacks(self):
    return self.total_callstacks

  def get_exclusive_rate(self):
    name_to_hits = {}
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        head = callstack[0]
        name_to_hits.setdefault(head, 0)
        name_to_hits[head] += 1
    total_callstacks = self.total_callstacks
    return defaultdict(lambda : 0, dict([(name, (hits / total_callstacks)) for name, hits in name_to_hits.items()]))

  def time_per_frame(self): 
    mean_frame_time = self.avg_frame_time()
    return defaultdict(lambda : 0, dict([(name, (mean_frame_time  * rate)) for name, rate in self.get_exclusive_rate().items()]))

  def function_names(self):
    result = []
    for callstacks in self.frame_index_to_callstacks.values():
      for callstack in callstacks:
        result += callstack
    return list(set(result))


In [16]:
states = pd.read_csv("~/cumbia_threadstates/mainthreadstates1.csv", header=None)
states.drop(0, inplace=True, axis=1)
states.drop(1, inplace=True, axis=1)
states = states.to_numpy()

frametrack_csv = pd.read_csv('~/cumbia_threadstates/frametrack1.csv').to_numpy()

In [2]:
def intersection(start1, end1, start2, end2):
  if start2 > end1 or start1 > end2:
    return 0
  return min(end1, end2) - max(start1, start2)

def get_running_durations(start, end, states):
  result = 0
  for row in states:
    result += intersection(start, end, row[0], row[1])
  return result


durations = []
for i in range(frametrack_csv.shape[0]):
  durations += [1 - get_running_durations(frametrack_csv[i, 2], frametrack_csv[i, 3], states)/ frametrack_csv[i, 4]]

NameError: name 'frametrack_csv' is not defined

In [31]:
len([d for d in durations if d > 0])

288

In [3]:
def load_capture_with_frametrack(callstack_csv_path, frametrack_csv_path, max_frametime, thread_running_path=None):
  frametrack_csv = pd.read_csv(frametrack_csv_path).to_numpy()
  frametrack_csv = frametrack_csv[frametrack_csv[:, 3].argsort()] # sort by end
  frame_durations = frametrack_csv[:, 4]
  frame_mean = frame_durations.mean()
  frame_ends = frametrack_csv[:, 3]
  last_init_frame = np.max((frame_durations > max_frametime).nonzero())
  init_end_time = frametrack_csv[last_init_frame, 3]

  frame_ends = frame_ends[last_init_frame:]

  if thread_running_path == None:
    frame_durations = frame_durations[last_init_frame:]
  else:
    thread_states = pd.read_csv(thread_running_path)

  frame_mean = frame_durations.mean()
  frame_var = frame_durations.var() / frame_durations[last_init_frame:].shape[0]
  
  return Capture(pd.read_csv(callstack_csv_path).to_numpy(), frame_mean, frame_var, frame_ends, init_end_time)

In [4]:
import json 

def load_cumbia_capture_with_results_json(callstack_csv_path, results_json_path):
  results = json.load(open(results_json_path))
  fps = results["outputs"][1]['values']
  fps = np.array(fps[1:])
  times = np.array(results["outputs"][1]['times'])
  times.sort()
  frame_mean = np.dot(np.diff(times), 1 / fps)/np.diff(times).sum()  * 1e9
  init_end_ns_from_start = times[1] * 1e9
  callstack_csv = pd.read_csv(callstack_csv_path).to_numpy()
  start_orbit_time = callstack_csv[:, 1].min()
  init_end_time = init_end_ns_from_start + start_orbit_time
  
  frame_ends = np.array([times.max() * 1e9 + start_orbit_time])
  return Capture(callstack_csv, frame_mean, frame_ends, init_end_time)
  

In [5]:
# capture1 -- slow
def test(capture1, rates1, capture2, rates2, name):
  n1 = capture1.total_callstacks
  n2 = capture2.total_callstacks
  p1 = rates1[name] 
  p2 = rates2[name]  
  f1 = capture1.avg_frame_time()
  f2 = capture2.avg_frame_time()

  stat = f1*p1 - f2*p2
  var_p1 = p1 * (1 - p1) / n1 
  var_p2 = p2 * (1 - p2) / n2
  var_f1 = capture1.frame_var
  var_f2 = capture2.frame_var 

  var1 = var_f1 * var_p1 + var_p1 * f1**2 + var_f1 * p1**2
  var2 = var_f2 * var_p2 + var_p2 * f2**2 + var_f2 * p2**2

  stat = stat / np.sqrt(var1 + var2)
  p = scipy.stats.norm().cdf(stat)
  if (np.isnan(p)): 
    return 1.0
  return min(p, 1-p) * 2

In [8]:
path = "~/cumbia_callstack_thread4k1.csv"
callstack = pd.read_csv(path).to_numpy()

In [14]:
diffs = np.diff(np.sort(callstack[:,1]))

In [32]:
import matplotlib.pyplot as plt
# n, bins, patches = plt.hist(diffs, 100, facecolor='blue')
# plt.show()
np.sum(diffs < 30000) , len(diffs)

(2, 391133)

In [6]:
# capture1 = load_cumbia_capture_with_results_json("~/cumbia_vsync_callstack1_thread.csv", '/usr/local/google/home/avanesov/result_vsync1.json')
# capture2 = load_cumbia_capture_with_results_json("~/cumbia_vsync_callstack2_thread.csv", '/usr/local/google/home/avanesov/result_vsync2.json')
capture1 = load_capture_with_frametrack("~/cumbia_callstack_thread4k1.csv", "~/cumbia_frametrack4k1.csv",  85*1e6)
capture2 = load_capture_with_frametrack("~/cumbia_callstack_thread4k2.csv", "~/cumbia_frametrack4k2.csv",  85*1e6)
# capture1 = load_capture_with_frametrack("~/cumbia_callstack7_thread.csv", "~/cumbia_frametrack7.csv",  85*1e6)
# capture2 = load_capture_with_frametrack("~/cumbia_callstack6_thread.csv", "~/cumbia_frametrack6.csv",  85*1e6)


names1 = capture1.function_names()
names2 = capture2.function_names()
names = set(names1).intersection(set(names2))
names.remove("???")
capture1.filter_functions(names)
capture2.filter_functions(names)

ParserError: Error tokenizing data. C error: Calling read(nbytes) on source failed. Try engine='python'.

In [60]:
capture1.frame_mean, capture2.frame_mean

(65741459.12401353, 65700346.80270575)

In [59]:
rate1 = capture1.get_exclusive_rate()
rate2 = capture2.get_exclusive_rate() 
pvalues = [(name, test(capture1, rate1, capture2, rate2, name)) for name in names]
reject, corrected,_,_ = multipletests([pvalue for name, pvalue in pvalues])

namesnp = np.array([name for name, pvalue in pvalues])
rejected_names = namesnp[np.array(reject)]


/tmp/ipykernel_3636859/2705725131.py:19: RuntimeWarning: invalid value encountered in double_scalars
  stat = stat / np.sqrt(var1 + var2)
/usr/local/google/home/avanesov/.local/lib/python3.9/site-packages/statsmodels/stats/multitest.py:177: RuntimeWarning: divide by zero encountered in log1p
  np.log1p(-pvals))


In [61]:
corrected[corrected < 0.05]

array([], dtype=float64)

In [62]:
len(rejected_names), len(names)

(0, 1327)

In [ ]:
[(name, rate1[name])  for name in rejected_names]

In [27]:
pvalues.sort(key = lambda x : x[1]) 


In [45]:
name = list(names)[10]
test(capture1, rate1, capture2, rate2, name)

1.1885236966089256e-11 7.1309209212431e-11 0.10429592994193262 0.04582583268481589 65.74145912401353 27.542569230732862


0.2908352704186795

In [ ]:
pvalues

In [ ]:
problem_name= pvalues[0][0]
rate1[problem_name], rate2[problem_name]

In [ ]:
test(capture1, rate1, capture2, rate2, problem_name)

In [ ]:
len(set(rejected_names).intersection(names_with_high_overhead))

In [ ]:
names_with_high_overhead

In [ ]:
def overhead_per_function(capture1, capture2):
  avg_frame_time1 = capture1.avg_frame_time()
  avg_frame_time2 = capture2.avg_frame_time()

  time_per_frame1 = capture1.time_per_frame()
  time_per_frame2 = capture2.time_per_frame()

  result = []
  for name in names:
    time1 = time_per_frame1[name]
    time2 = time_per_frame2[name]
    result.append((name, (time1 - time2) / (avg_frame_time1 - avg_frame_time2)))
  return result

overheads = overhead_per_function(capture1, capture2)
overheads.sort(key = lambda x: -x[1])
overheads_dict = dict(overheads)
names_with_high_overhead = [name for name in names if overheads_dict[name] > 0.01]

In [20]:
frametrack1_path = '~/cumbia_frametrack8.csv'
frametrack2_path = '~/cumbia_frametrack9.csv'
frametrack1 = pd.read_csv(frametrack1_path).to_numpy()[:, 4]
frametrack2 = pd.read_csv(frametrack2_path).to_numpy()[:, 4]

In [21]:
frametrack1

array([2353792, 311437, 763061, ..., 14007796, 14562252, 914249],
      dtype=object)

In [22]:
import scipy
from scipy.stats import ttest_ind

In [23]:
ttest_ind(frametrack1, frametrack2, equal_var=False)

Ttest_indResult(statistic=0.625311070682517, pvalue=0.531799155167028)

In [24]:
frametrack1.mean(), frametrack2.mean()

(26442199.976902176, 26235668.44)